In [1]:
import numpy as np
import torch
import torch.nn as nn

In [2]:
# Training data
# [Temp, Rainfall, Humidity]
inputs = np.array([[73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70],
                   [73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70],
                   [73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70]],dtype='float32')
# [Apples, Oranges]
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,119],
                    [56,70],[81,101],[119,133],[22,37],[103,119],
                    [56,70],[81,101],[119,133],[22,37],[103,119]], dtype='float32')

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [4]:
# We have 15 training examples here
from torch.utils.data import TensorDataset
# We will create a TensorDataset which will allow access to rows from inputs and targets as tuples, 
# and provides standard APIs for working with different types of datasets in Pytorch.

In [5]:
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [6]:
# We will create a Dataloader, which can split data into batches of a predefined size while training. It also provides other features like shuffling and sampling of data.

In [7]:
from torch.utils.data import DataLoader

In [8]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle = True)

In [9]:
i = 0
for xb,yb in train_dl:
  i += 1
  print("Batch",i)
  print(xb)
  print(yb)

Batch 1
tensor([[69., 96., 70.],
        [91., 88., 64.],
        [69., 96., 70.],
        [91., 88., 64.],
        [69., 96., 70.]])
tensor([[103., 119.],
        [ 81., 101.],
        [103., 119.],
        [ 81., 101.],
        [103., 119.]])
Batch 2
tensor([[102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.]])
tensor([[ 22.,  37.],
        [119., 133.],
        [119., 133.],
        [119., 133.],
        [ 22.,  37.]])
Batch 3
tensor([[ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]])
tensor([[ 56.,  70.],
        [ 56.,  70.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.]])


In [14]:
# Define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0591,  0.1334, -0.2160],
        [ 0.1321,  0.3749,  0.5753]], requires_grad=True)
Parameter containing:
tensor([ 0.4034, -0.1892], requires_grad=True)


In [15]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0591,  0.1334, -0.2160],
         [ 0.1321,  0.3749,  0.5753]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4034, -0.1892], requires_grad=True)]

In [16]:
preds = model(inputs)
print(preds)

tensor([[ 4.3723, 59.3068],
        [ 3.7032, 81.6374],
        [10.9011, 94.9013],
        [ 4.1797, 50.6893],
        [ 2.1745, 85.1817],
        [ 4.3723, 59.3068],
        [ 3.7032, 81.6374],
        [10.9011, 94.9013],
        [ 4.1797, 50.6893],
        [ 2.1745, 85.1817],
        [ 4.3723, 59.3068],
        [ 3.7032, 81.6374],
        [10.9011, 94.9013],
        [ 4.1797, 50.6893],
        [ 2.1745, 85.1817]], grad_fn=<AddmmBackward0>)


In [17]:
import torch.nn.functional as F

In [18]:
loss_fn = F.mse_loss

In [19]:
# ?nn.Linear => Help function to see/ visualize the model and its parameters

In [20]:
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(3408.0784, grad_fn=<MseLossBackward0>)


In [21]:
opt = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [22]:
def fit(num_epochs, model, loss_fn, opt):
  # Repeat for couple number of epochs
  for epoch in range(num_epochs):
    for xb,yb in train_dl:
      # Generate a prediction
      preds = model(xb)
      # Calculate the loss value
      loss = loss_fn(preds,yb)
      # Compute gradients
      loss.backward()
      # Update parameters using gradients
      opt.step()
      # Reset the gradients to zero
      opt.zero_grad()
    # Print the progress
    if (epoch+1) % 10 == 0:
      print(' Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs,loss.item()))

In [23]:
fit(100, model, loss_fn, opt)

 Epoch [10/100], Loss: 239.8796
 Epoch [20/100], Loss: 92.3985
 Epoch [30/100], Loss: 14.9129
 Epoch [40/100], Loss: 74.4812
 Epoch [50/100], Loss: 7.6514
 Epoch [60/100], Loss: 28.1430
 Epoch [70/100], Loss: 24.3429
 Epoch [80/100], Loss: 48.3902
 Epoch [90/100], Loss: 26.8884
 Epoch [100/100], Loss: 21.7151


In [25]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1981,  0.9408,  0.2224],
         [-0.1913,  0.8014,  0.7340]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4041, -0.1908], requires_grad=True)]